In [ ]:
import pandas as pd 
import numpy as np
import yfinance as yf
from datetime import datetime as dt
from plotnine import *
from mizani.formatters import percent_format

# define list of tickers
tickerlist = ['UNH', 'MSFT', 'GS', 'HD', 'CAT', 'CRM', 'MCD',
               'V', 'AMGN', 'TRV', 'AXP', 'BA', 'HON', 'JPM',
                 'IBM', 'AAPL', 'AMZN', 'JNJ', 'PG', 'CVX',
                   'MRK', 'DIS', 'NKE', 'MMM', 'KO', 'WMT', 
                   'DOW', 'CSCO', 'INTC', 'VZ']
return_col = 'log_return' # alternatively use 'ret' to get standard returns

In the following we complete the exercises for mandatory assignment 1.

**Exercise 1. Compute monthly returns for each of the tickers.**

In [ ]:
inputdf = pd.read_pickle(r'DJ_comp.pkl') # inputdf = yf.download(tickers=tickerlist,period='25y')
# transform the dataset
df = (inputdf
      .get('Adj Close')
      .stack()
      .reset_index()
      .rename(columns={'Date':'date','level_1':'ticker',0:'adj_close'})
      .set_index('date')
      .groupby('ticker')['adj_close']
      .resample('m')
      .last()
      .reset_index()
      .assign(date=lambda x: x['date'].dt.strftime('%Y-%m-%d'))  # Convert date to string
      .query("date >= '2000-01-01'")  # Filter for dates >= 1999-04-01
      .query("date <= '2023-12-31'")  # Filter for dates >= 1999-04-01
      )

# remove all tickers where there is no data for each month 
df = (df
  .groupby(["ticker"])
  .apply(lambda x: x.assign(counts=x["adj_close"].dropna().count()))
  .reset_index(drop=True)
  .query("counts == counts.max()")
  .drop(columns='counts')
  .assign(ret=lambda x: x["adj_close"].pct_change())
  .assign(log_return = lambda x: np.log(x['adj_close']/x['adj_close'].shift(1)) )
)

# check that we have 27 tickers
no_of_tickers = int(df['ticker'].unique().size)
#print(f'The number of tickers in the dataset: { no_of_tickers}')

#
mu = np.array(df.groupby('ticker')[return_col].mean()).T
sigma = np.array(df.pivot(index='date',columns='ticker',values=return_col).cov())

We load data for a 25 year period for all constituents of the Dow Jones 30 Index. The dataset is cleaned by removing all consituents that don't have price data for each trading day in the period January 1st 2000 to December 31st 2023. `{python} len(inputdf['ticker'].unique())-len(df['ticker'].unique())` constituents are removed due to no price data. 
Further, we calculate the monthly return for each ticker in the dataset. Here, we have chosen to use `{python} if return_col == 'log_return': 'log return' else: 'standard return'`

**Exercise 2. Which of the 𝑁 individual assets delivered the highest Sharpe ratio?**

In [ ]:
# Calculate Sharpe ratio for each stock
sharpe_ratio = (mu - 0) / np.diag(sigma)
# Print the Sharpe ratio
sr_high = df.groupby('ticker')['ticker'].first().iloc[sharpe_ratio.argmax()]

Based on the monthly return series calculated earlier, we create a vector of the average monthly returns $\mu$ for each series and a variance-covariance matrix $\Sigma$. To calculate the Sharpe-ratio, we use the formula: $Sharpe=\frac{return-r_f}{standarddeviation}$. The stock with the highest Sharpe ratio is `{python} sr_high` with a ration of `{python} sharpe_ratio.max().round(2)`.


In [ ]:
def calc_return_std(weights, mu, sigma_matrix, factor):
  """
  Calculate the expected return and standard deviation of a portfolio.

  Parameters:
  weights (numpy.ndarray): The weights of the assets in the portfolio.
  mu (numpy.ndarray): The expected returns of the assets.
  sigma_matrix (numpy.ndarray): The covariance matrix of the assets.
  factor (float): A scaling factor.

  Returns:
  tuple: A tuple containing the expected return and standard deviation of the portfolio.
  """
  return_vec = mu.T @ weights * factor 
  vol = np.sqrt(weights.T @ sigma_matrix @ weights) * np.sqrt(factor)
  return return_vec, vol

**Exercise 3. Perform each of the following steps:**
First of we create a function compute_efficient_frontier which return an object with the inputs that consists of a vector of the estimated expected return, the estimated variance-covariance matrix and a factor that is used to annualised the return, minimum variance portfolio, efficient portfolio that delivers two times the expected return of the minimum variance portfolio and lastly the efficient frontier. 
*How are the different things calculated*
***Compute the minimum variance portfolio weight**
To compute the minimum variance portfolio we perform


In [ ]:
def compute_efficient_frontier(mu_est: np.array, sigma_est: np.array, yearly_factor: int=1) -> pd.DataFrame:
    """
    The function performs each of the following steps:
    1. Compute the minimum variance portfolio weight mvp_weights for the input Sigma_est. 
        The function can handle positive definite variance-covariance matrices of arbitrary dimensions N x N.
    2. Compute the eﬀicient portfolio weights efp_weights for the inputs Sigma_est and mu_est 
      that delivers two times the expected return of the minimum variance portfolio weight.
    3. Make use of the two-mutual fund theorem to characterize a range of portfolio weights on the eﬀicient frontier: 
      Specifically, compute the weights of a sequence of portfolios which are combinations of the minimum variance portfolio weight
        and the eﬀicient portfolio, c_weights = c * mvp_weights  + (1 - c) * efp_weights where c ∈ {-0.1, ... , 1.2}
    
      Args: 
        mu_est (np.array): N x 1 return matrix
        sigma_est (np.array): N x N variance-covariance matrix
      Returns: 
        df: with column c which is the weight of the minimum variance portfolio and N corresponding columns with the weights 
          of each asset
    """
    class results: pass
    results.inputs = {}
    results.inputs['mu'] = mu_est
    results.inputs['sigma'] = sigma_est
    results.inputs['yearly_factor'] = yearly_factor

    N = mu_est.shape[0]
    if sigma_est.shape[0] != N: 
        raise ValueError('The size length of vector mu_est should be the same as for sigma_est')
    iota = np.ones(N)
    sigma_inv = np.linalg.inv(sigma_est) 

    #----- minimum variance portfolio
    mvp_weights = sigma_inv @ iota
    mvp_weights = mvp_weights/mvp_weights.sum()
    mvp_return,mvp_volatility =  calc_return_std(mvp_weights,mu,sigma_est,yearly_factor)
    print(f'Return of the minimum variance portfolio is: {mvp_return:.2f} and its volatility is {mvp_volatility:.2f}')
    results.mvp_weights = mvp_weights  # store minimum variance portfolio weights in object

    #----- efficient frontier portfolio
    mu_bar = mvp_return*2
    C = iota.T @ sigma_inv @ iota
    D = iota.T @ sigma_inv @ mu_est
    E = mu_est.T @ sigma_inv @ mu_est
    lambda_tilde = 2 * (mu_bar - D/C) / (E-D**2/C)
    efp_weights = mvp_weights + lambda_tilde/2 * (sigma_inv@mu_est - D* mvp_weights ) 
    efp_return,efp_vol =  calc_return_std(efp_weights,mu,sigma_est,yearly_factor)
    print(f'Return of the efficient frontier portfolio is: {efp_return:.2f} and its volatility is {efp_vol:.2f}')
    results.efp_weights = efp_weights # store efficient frontier portfolio weights in object

    #----- mutual fund theorem
    a = np.linspace(-0.2, 1.2, 121)
    res = pd.DataFrame(columns=["mu", "sd"], index=a).astype(float)
    for i in a:
        w = i*mvp_weights+(1-i)*efp_weights
        for j in range(len(w)):
          res.loc[i, f"w_{j+1}"] = w[j]  # Assign each element of w to a named column
        res.loc[i, "mu"] = (w.T @ mu)*yearly_factor
        res.loc[i, "sd"] = np.sqrt(w.T @ sigma @ w)*np.sqrt(yearly_factor)
    
    results.res = res   # store dataframe in object 

    return results

In [ ]:
cef = compute_efficient_frontier(mu_est=mu,sigma_est=sigma,yearly_factor=12)

efpRet, efpVol = calc_return_std(weights=cef.efp_weights,mu=cef.inputs['mu'],sigma_matrix=cef.inputs['sigma'],factor=cef.inputs['yearly_factor'])
efpSR = efpRet/efpVol

mvpRet, mvpVol = calc_return_std(weights=cef.mvp_weights,mu=cef.inputs['mu'],sigma_matrix=cef.inputs['sigma'],factor=cef.inputs['yearly_factor'])
mvpSR = mvpRet/mvpVol

The minimum variance portfolio has a expected return of `{python} (mvpRet*100).round(2)` and a volatility of `{python} (mvpVol*100).round(2)`. While the volatility of the efficient portfolio that delivers two times the expected return of the minimum variance is `{python} (efpVol*100).round(2)`. We notice that the Sharpe ratio `{python} if mvpSR > efpSR: 'decreases' else: 'increases'` from `{python} mvpSR` to `{python} efpSR`. 


In [ ]:
#---- tangency portfolio
# ¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿
#       connsider if there is a better way
# ????????????????????????????????????????????????
d = np.vstack((cef.inputs['sigma'],np.ones(27)))
c = np.append(cef.inputs['mu'],0)
e = np.hstack((d,c.reshape(c.shape[0],1)))
q = np.append(np.zeros(cef.inputs['mu'].shape[0]),1)
tan_weights = np.linalg.inv(e) @ q
tan_weights = tan_weights[:-1]
tanRet, tanVol = calc_return_std(weights=tan_weights,mu=cef.inputs['mu'],sigma_matrix=cef.inputs['sigma'],factor=cef.inputs['yearly_factor'])

# sharpe-ratio
sharpeRatio = tanRet/tanVol

vol = np.linspace(0,tanVol,101)
x = np.linspace(0,1,101)
ret = np.nan + np.zeros(vol.shape[0])
for i, v in enumerate(x):
   ret[i] = tanRet * v
tanLine = pd.DataFrame((vol,ret),index=['vol','ret']).T

res_figure = (
  ggplot(cef.res, aes(x="sd", y="mu")) +
  geom_point() + 
  geom_line(tanLine, aes(x='vol',y='ret'))+
  geom_point(
    pd.DataFrame({"mu": [mvpRet, efpRet,tanRet],"sd":[mvpVol, efpVol,tanVol]}),
    size=4, 
    color='darkblue',
    ) +
  geom_point(
    pd.DataFrame({"mu": cef.inputs['mu']*12,
                  "sd": np.sqrt(np.diag(cef.inputs['sigma'])) * np.sqrt(12)
                  })
  ) +
  labs(x="Annualized standard deviation",
       y="Annualized expected return",
       title="Efficient frontier for DOW index constituents") +
  scale_x_continuous(labels=percent_format()) +
  scale_y_continuous(labels=percent_format())
)

In [ ]:
res_figure

**Exercise 4. Efficient tangency:**
*What are the eﬀicient tangency portfolio weights 𝜔tgc under the assumption that the risk- free rate is zero based on the true parameters 𝜇 and Σ? Do the tangency portfolio weights seem like a well-balanced portfolio? What are the potential issues when implementing this portfolio in reality? What is the maximum attainable Sharpe ratio assuming the risk-free rate is zero? Should the Sharpe ratio of 𝜔tgc be higher or lower than the Sharpe ratio of the individual assets?*



**Exercise 4. Explain the code:**